Source: https://arxiv.org/pdf/2210.06774.pdf

Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [ ]:
%pip install openai -- upgrade

In [ ]:
# Get the openai secret key
import getpass
from openai import OpenAI

secret_key = getpass.getpass("Please enter your openai key: ")

In [ ]:
# OpenAI set up:
client = OpenAI()
client.api_key = secret_key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

In [ ]:
# Add your premise:
premise_input = "A new law grad returns home to start her career, but struggles with the broken justice system." #@param {type:"string"}
premise = f"Premise: {premise_input}"
premise

## Plan Module

In [ ]:
# Setting:
setting_prompt = f"{premise}\nThe story is set in"
setting_response = complete(setting_prompt, stop=["."])
setting = f"Setting: The story is set in {setting_response}."
setting

In [ ]:
# Characters
num_characters = 2
characters = ""
character_format = "Please invent characters for this story, in the format of [name] is [context]. For example 'Liza Turner is a 22-year-old woman' or 'Peyton Turner is Liza's older sister'. Each character is distinct and does not reuse the same name as these examples."
character_prompt = f"Premise: {premise}\n\nSetting: {setting}\n\n{character_format}\n\n"
for n in range(num_characters):
    character_prompt += f"{n+1}. Character Portrait:\n"
    character_response = complete(f"{character_prompt}\n\n{n+1}. Character Portrait:\n", stop=["."])
    character_prompt += f"{character_response}\n\n"
    characters += f"{n+1}. Character Portrait:\n{character_response.strip()}.\n\n"

print(characters)

In [ ]:
# Plot
plot_prompt = f"{premise}\n\n{setting}\n\n{characters}Outline the main plot points of the story.\n1."
plot_response = complete(plot_prompt)
plot = f"{plot_response}"
print("Outline the main plot points of the story\n\n1. ", plot)

In [ ]:
# Full Plan Module
print(f"{premise}\n\n{setting}\n\n{characters}\n\nOutline the main plot points of the story\n\n1. {plot}")

## Draft Module

In [ ]:
import re

plot_points_raw = plot.split('\n')
plot_points = []

for pp in plot_points_raw:
    if pp == '':
        continue
    else:
        stripped_pp = re.sub(r'^\d+\.\s*', '', pp)
        plot_points.append(stripped_pp)

print(plot_points)

In [ ]:
drafts = []

for idx in range(len(plot_points)):
    if idx == 0:
        previous = ""
        immediately = ""
    else:
        previous = f"Previous story summary:\n{plot_points[idx-1]}"
        immediately = f"Immediately before the current passage:\n{drafts[idx-1]}"

    upcoming = f"In the upcoming passage,\n{plot_points[idx]}"

    draft_prompt = f"Relevant context:\n{characters}{previous}\n\n{immediately}\n\n{upcoming}\n\nFull text below:"

    draft_response = complete(draft_prompt)
    draft_response = draft_response.strip()

    print(f"Plot Point {idx+1}\n-----\n", draft_prompt, f"\n{draft_response}\n-----\n\n")
    drafts.append(draft_response)

In [ ]:
print("".join(drafts))

## Revision

In [ ]:
# Progressive extraction
facts = []
edits = []
for idx in range(1, len(drafts)):
    # Old Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx-1]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()
    facts.append(facts_response)

    # New Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()

    # Consistency
    joined_facts = "\n".join(facts)
    consistency_prompt = f"Are any New Facts inconsistent with Old Facts?:\n\Old facts:\n{joined_facts}\n\nNew facts:{facts_response}\n\nInconsistencies:\n-"
    consistency_response = complete(consistency_prompt)

    # Rewriting
    edit_prompt = f"Edit so that any inconsistencies are addressed:\nInconsistencies:{consistency_response}\n\nText:{drafts[idx]}\n\nRewritten Text:"
    edit_response = complete(edit_prompt)
    edit_response = edit_response.strip()
    edits.append(edit_response)

    print(f"Draft Text {idx+1}\n-----\n", drafts[idx], f"\n\nOld Facts:\n{joined_facts}", f"\n\nNew Facts:\n{facts_response}", f"\n\nConsistency:\n{consistency_response}", f"\n\nRewritten Text:\n{edit_response}\n-----\n\n")


In [ ]:
"/n".join(edits)